In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import glob
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing import image
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D

In [ ]:
from glob import glob
train_images = glob('/kaggle/input/denoising-dirty-documents/train/*.png')
train_labels = glob('/kaggle/input/denoising-dirty-documents/train_cleaned/*.png')
test_images = glob('/kaggle/input/denoising-dirty-documents/test/*.png')
print("Total number of images in the training set: ", len(train_images))
print("Total number of cleaned images found: ", len(train_labels))
print("Total number of samples in the test set: ", len(test_images))

### Sample of train images and train labels

In [ ]:
samples = train_images[:3] + train_labels[:3]

f, ax = plt.subplots(2,3,figsize = (20,10))
for i, img in enumerate(samples):
    img = cv2.imread(img)
    ax[i//3, i%3].imshow(img, cmap='gray')
    ax[i//3, i%3].axis('off')
plt.show()  

## CV2 Method

In [ ]:
img = cv2.imread(train_images[0], cv2.IMREAD_GRAYSCALE)
plt.imshow(img,cmap = plt.cm.gray)
plt.show()

In [ ]:
# applying different thresholding  
# techniques on the input image 
thresh1 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 30) 
plt.imshow(thresh1,cmap = plt.cm.gray) 
# plt.imshow(thresh2,cmap = plt.cm.gray) 
plt.show()

In [ ]:
# applying different thresholding  
# techniques on the input image 
thresh2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 25) 
plt.imshow(thresh2,cmap = plt.cm.gray) 
plt.show()

In [ ]:
from PIL import Image
# cvt_image = cv2.cvtColor(thresh2, cv2.COLOR_GRAY2RGB)
im_pil = Image.fromarray(thresh2)
im_resized = im_pil.resize((540,420))
im_array = image.img_to_array(im_resized)

im_array.shape

In [ ]:
im_resized

In [ ]:
proc_imgs = []
for k, figs in enumerate(zip(train_images, train_labels)):
    train_img = cv2.imread(figs[0], cv2.IMREAD_GRAYSCALE)
    proc_img = cv2.adaptiveThreshold(train_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 25)
    
    # cvt_image = cv2.cvtColor(proc_img, cv2.COLOR_GRAY2RGB)
    im_pil = Image.fromarray(proc_img)
    im_resized = im_pil.resize((540,420))
    # plt.imshow(im_resized,cmap=plt.cm.gray)
    # plt.show()
    im_array = image.img_to_array(im_resized).astype('float32')/255
    
    proc_imgs.append(im_array)

## AutoEncoder

In [ ]:
def AutoEncoder():
    input_img = Input(shape=(420,540,1), name = 'input_image')
    
    # Encoder Layer
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv1')(input_img)
    x = MaxPooling2D((2,2), padding='same', name='pool1')(x)
    
    # Decoder
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv2')(x)
    x = UpSampling2D((2,2), name='upsample3')(x)
    x = Conv2D(1, (3,3), activation='sigmoid', padding='same', name='Conv3')(x)
    
    ae = Model(inputs=input_img, outputs=x)
    ae.compile(optimizer='Adagrad', loss='binary_crossentropy')
    return ae

In [ ]:
ae = AutoEncoder()
ae.summary()

In [ ]:
X = []
Y = []

for img in train_images:
    img = image.load_img(img, grayscale=True,target_size=(420,540))
    img = image.img_to_array(img).astype('float32')/255
    X.append(img)

for img in train_labels:
    img = image.load_img(img, grayscale=True,target_size=(420,540))
    img = image.img_to_array(img).astype('float32')/255
    Y.append(img)


X = np.array(X)
Y = np.array(Y)

print("Size of X : ", X.shape)
print("Size of Y : ", Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and validation. Always set the random state!!
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=100)
print("Total number of training samples: ", X_train.shape)
print("Total number of validation samples: ", X_valid.shape)

In [ ]:
# Train your model
ae.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_valid, y_valid))

In [ ]:
sample_test = image.load_img(test_images[10], grayscale=True, target_size=(420,540))
sample_test = image.img_to_array(sample_test)
sample_test_img = sample_test.astype('float32')/255.
sample_test_img = np.expand_dims(sample_test, axis=0)

# Get the predition
predicted_label = np.squeeze(ae.predict(sample_test_img))

f, ax = plt.subplots(1,2, figsize=(10,8))
ax[0].imshow(np.squeeze(sample_test), cmap='gray')
ax[1].imshow(np.squeeze(predicted_label.astype('int8')), cmap='gray')
plt.show()